<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Federated_Learning_(FL).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install syft

In [ ]:
pip show syft

In [ ]:
pip install syft==0.2.9

In [ ]:
import syft as sy
import torch
import torch.nn as nn
import torch.optim as optim

# Setup PySyft hook and workers
hook = sy.TorchHook(torch)
alice = sy.VirtualWorker(hook, id="alice")
bob = sy.VirtualWorker(hook, id="bob")

# Define the model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 2)

    def forward(self, x):
        return self.fc1(x)

# Initialize model and send to workers
model = Net()
model_alice = model.send(alice)
model_bob = model.send(bob)

# Prepare data for Alice and Bob
data_alice = torch.tensor([[1.0, 1.0], [2.0, 2.0]], requires_grad=True).send(alice)
target_alice = torch.tensor([[0.0], [1.0]], requires_grad=True).send(alice)

data_bob = torch.tensor([[3.0, 3.0], [4.0, 4.0]], requires_grad=True).send(bob)
target_bob = torch.tensor([[1.0], [0.0]], requires_grad=True).send(bob)

# Define criterion and optimizer
criterion = nn.MSELoss()
optimizer_alice = optim.SGD(params=model_alice.parameters(), lr=0.1)
optimizer_bob = optim.SGD(params=model_bob.parameters(), lr=0.1)

# Training loop
for epoch in range(10):
    # Train on Alice's data
    optimizer_alice.zero_grad()
    output_alice = model_alice(data_alice)
    loss_alice = criterion(output_alice, target_alice)
    loss_alice.backward()
    optimizer_alice.step()

    # Train on Bob's data
    optimizer_bob.zero_grad()
    output_bob = model_bob(data_bob)
    loss_bob = criterion(output_bob, target_bob)
    loss_bob.backward()
    optimizer_bob.step()

    # Fetch and log loss
    print(f"Epoch {epoch}, Loss Alice: {loss_alice.get():.4f}, Loss Bob: {loss_bob.get():.4f}")

# Retrieve updated models
model_alice = model_alice.get()
model_bob = model_bob.get()

In [ ]:
import syft as sy
import torch
import torch.nn as nn
import torch.optim as optim

# Create a PySyft domain and device
domain = sy.Domain(name="example_domain")
device = domain.add_device(sy.Device(name="alice_device"))

# Define model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 2)

    def forward(self, x):
        return self.fc1(x)

model = Net()
model.to(device)

# Create data and target tensors
data = torch.tensor([[1.0, 1.0], [2.0, 2.0]])
target = torch.tensor([[0.0], [1.0]])

# Send data and target to device
data_ptr = data.send(device)
target_ptr = target.send(device)

# Define optimizer and loss function
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# Training loop
for epoch in range(10):
    optimizer.zero_grad()
    output = model(data_ptr)
    loss = criterion(output, target_ptr)
    loss.backward()
    optimizer.step()
    print(f'Epoch {epoch}, Loss: {loss.get()}')